<a href="https://colab.research.google.com/github/HoaiNhu/MachineLearning_Text/blob/main/ML_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sunilthite/text-document-classification-dataset")

print("Path to dataset files:", path)

100%|██████████| 1.85M/1.85M [00:00<00:00, 38.4MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/sunilthite/text-document-classification-dataset/versions/1


# **Tiền xử lí dữ liệu**

Đọc dữ liệu

In [3]:
import os

# Kiểm tra các tệp trong thư mục tải về
print("Các tệp trong thư mục dữ liệu:")
print(os.listdir(path))


Các tệp trong thư mục dữ liệu:
['df_file.csv']


In [4]:
import os

# Liệt kê các tệp trong thư mục
dataset_path = '/root/.cache/kagglehub/datasets/sunilthite/text-document-classification-dataset/versions/1'
print("Các tệp có trong thư mục:")
print(os.listdir(dataset_path))


Các tệp có trong thư mục:
['df_file.csv']


In [5]:
# Tạo liên kết thư mục để dễ truy cập
!ln -s /root/.cache/kagglehub/datasets/sunilthite/text-document-classification-dataset/versions/1 /content/dataset

# Kiểm tra lại trong cây thư mục /content
!ls /content/dataset


df_file.csv


In [6]:
import pandas as pd

# Đường dẫn tới file CSV (cập nhật đúng đường dẫn của bạn)
file_path = '/root/.cache/kagglehub/datasets/sunilthite/text-document-classification-dataset/versions/1/df_file.csv'
# Đọc dữ liệu
df = pd.read_csv(file_path)

# Kiểm tra dữ liệu
print("Kích thước dữ liệu:", df.shape)
print(df.head())


Kích thước dữ liệu: (2225, 2)
                                                Text  Label
0  Budget to set scene for election\n \n Gordon B...      0
1  Army chiefs in regiments decision\n \n Militar...      0
2  Howard denies split over ID cards\n \n Michael...      0
3  Observers to monitor UK election\n \n Minister...      0
4  Kilroy names election seat target\n \n Ex-chat...      0


# **Tiền xử lí văn bản**

In [7]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Tải các công cụ NLP
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Hàm tiền xử lý văn bản
def preprocess_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Loại bỏ ký tự đặc biệt
    text = text.lower()  # Chuyển thành chữ thường
    text = ' '.join([lemmatizer.lemmatize(word) for word in text.split() if word not in stop_words])
    return text

# Áp dụng tiền xử lý
df['Text'] = df['Text'].apply(preprocess_text)
print("Dữ liệu sau khi tiền xử lý:")
print(df.head())


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Dữ liệu sau khi tiền xử lý:
                                                Text  Label
0  budget set scene election gordon brown seek pu...      0
1  army chief regiment decision military chief ex...      0
2  howard denies split id card michael howard den...      0
3  observer monitor uk election minister invite i...      0
4  kilroy name election seat target exchat show h...      0


# **Chia tập dữ liệu**

In [8]:
from sklearn.model_selection import train_test_split

# Chia dữ liệu
X_train, X_test, y_train, y_test = train_test_split(df['Text'], df['Label'], test_size=0.2, random_state=42)

print("Số lượng mẫu trong tập huấn luyện:", len(X_train))
print("Số lượng mẫu trong tập kiểm tra:", len(X_test))


Số lượng mẫu trong tập huấn luyện: 1780
Số lượng mẫu trong tập kiểm tra: 445


# **Biểu diễn văn bản bằng TF-IDF**

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Tạo vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)  # Giới hạn từ điển tối đa 5000 từ

# Biểu diễn TF-IDF
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("Kích thước của tập dữ liệu TF-IDF:", X_train_tfidf.shape)


Kích thước của tập dữ liệu TF-IDF: (1780, 5000)


# **Huấn luyện mô hình**

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

# Khởi tạo và huấn luyện mô hình
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test_tfidf)

# Đánh giá kết quả
print("Độ chính xác:", accuracy_score(y_test, y_pred))
print("Báo cáo phân loại:\n", classification_report(y_test, y_pred))


Độ chính xác: 0.9662921348314607
Báo cáo phân loại:
               precision    recall  f1-score   support

           0       0.95      0.96      0.95        92
           1       0.99      1.00      0.99        98
           2       0.96      0.96      0.96        77
           3       0.96      0.96      0.96        76
           4       0.97      0.95      0.96       102

    accuracy                           0.97       445
   macro avg       0.97      0.97      0.97       445
weighted avg       0.97      0.97      0.97       445



# **Thử nghiệm với dữ liệu mới**

In [19]:
# new_text = "The government announced new policies for economic growth."
# new_text = "Manchester United won the game last night with a stunning performance."
# new_text = "The stock market saw a sharp rise after positive economic reports."
new_text = "Apple releases the latest iPhone model with advanced AI features."
cleaned_text = preprocess_text(new_text)
vectorized_text = tfidf_vectorizer.transform([cleaned_text])

# Dự đoán
predicted_label = model.predict(vectorized_text)
categories = {0: "Politics", 1: "Sport", 2: "Technology", 3: "Entertainment", 4: "Business"}
print("Văn bản thuộc chủ đề:", categories[predicted_label[0]])


Văn bản thuộc chủ đề: Technology


In [ ]:
# Kiểm tra tổng số dòng trong DataFrame
print("Tổng số dòng trong dữ liệu:", len(df))

# Kiểm tra các dòng đầu và cuối
print("5 dòng đầu tiên:")
print(df.head())
print("5 dòng cuối cùng:")
print(df.tail())


Tổng số dòng trong dữ liệu: 2225
5 dòng đầu tiên:
                                                Text  Label
0  budget set scene election gordon brown seek pu...      0
1  army chief regiment decision military chief ex...      0
2  howard denies split id card michael howard den...      0
3  observer monitor uk election minister invite i...      0
4  kilroy name election seat target exchat show h...      0
5 dòng cuối cùng:
                                                   Text  Label
2220  india open sky competition india allow domesti...      4
2221  yukos bankruptcy u matter russian authority ab...      4
2222  survey confirms property slowdown government f...      4
2223  high fuel price hit ba profit british airway b...      4
2224  u trade gap hit record 2004 gap u export impor...      4


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

# Kiểm tra xem thư mục có tồn tại không
dataset_path = '/root/.cache/kagglehub/datasets/sunilthite/text-document-classification-dataset/versions/dataset_text'
print("Dữ liệu trong thư mục:", os.listdir(dataset_path))


Dữ liệu trong thư mục: ['df_file.csv']


In [ ]:
# Đường dẫn đến thư mục Google Drive để lưu dữ liệu
save_path = '/content/drive/My Drive/Colab Notebooks/dataset_text/'

# Nếu thư mục chưa tồn tại, tạo thư mục mới
os.makedirs(save_path, exist_ok=True)

# Di chuyển tất cả tệp trong dataset vào thư mục Google Drive
import shutil

for file_name in os.listdir(dataset_path):
    full_file_name = os.path.join(dataset_path, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, save_path)

print(f"Dữ liệu đã được sao chép vào: {save_path}")


Dữ liệu đã được sao chép vào: /content/drive/My Drive/Colab Notebooks/dataset_text/
